# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [67]:
import pandas as pd
import requests

# 1)
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
m, n = sinasc.drop_duplicates().shape
print(f"{m} linhas,", f"e {n} colunas.")

(27028, 69)
27028 linhas, e 69 colunas.


> Não há linhas duplicadas

In [68]:
sinasc.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

In [69]:
# 2) dados faltantes por variavel
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [70]:
# 3) seleciona colunas de interesse e conta dados faltantes
filtro = ["LOCNASC","IDADEMAE","ESTCIVMAE","ESCMAE","QTDFILVIVO","GESTACAO","GRAVIDEZ","CONSULTAS","APGAR5"]
sinasc_red = sinasc[filtro].copy()
sinasc_red.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [97]:
# 4) extrair apenas APGAR5 preenchido
sinasc_apgar5 = sinasc_red[~sinasc_red['APGAR5'].isna()].copy()
sinasc_apgar5.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [98]:
# 5) seu código aqui
# sinasc_apgar5[sinasc_apgar5['ESTCIVMAE'] == 9]
# sinasc_apgar5[['ESTCIVMAE','CONSULTAS']].value_counts()
sinasc_apgar5.ESTCIVMAE = sinasc_apgar5.ESTCIVMAE.fillna(9)
sinasc_apgar5.CONSULTAS = sinasc_apgar5.CONSULTAS.fillna(9)
sinasc_apgar5.ESTCIVMAE = sinasc_apgar5.ESTCIVMAE.astype('int64')
sinasc_apgar5.CONSULTAS = sinasc_apgar5.CONSULTAS.astype('int64')

sinasc_apgar5.info()
sinasc_apgar5.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 26925 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     26925 non-null  int64  
 1   IDADEMAE    26925 non-null  int64  
 2   ESTCIVMAE   26925 non-null  int64  
 3   ESCMAE      26615 non-null  object 
 4   QTDFILVIVO  25359 non-null  float64
 5   GESTACAO    25709 non-null  object 
 6   GRAVIDEZ    26849 non-null  object 
 7   CONSULTAS   26925 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 2.1+ MB


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [99]:
# 6) Seu código aqui
sinasc_apgar5.QTDFILVIVO = sinasc_apgar5.QTDFILVIVO.fillna(0)
sinasc_apgar5.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [108]:
# 7) seu código aqui
sinasc_apgar5.ESCMAE = sinasc_apgar5.ESCMAE.fillna('Ignorado')
sinasc_apgar5.GESTACAO = sinasc_apgar5.GESTACAO.fillna('Ignorado')
sinasc_apgar5.GRAVIDEZ = sinasc_apgar5.GRAVIDEZ.fillna('Ignorado')

In [106]:
# mapp = sinasc_apgar5["ESCMAE"].map({
#     'Nenhuma':1,
#     '1 a 3 anos':2,
#     '4 a 7 anos':3,
#     '8 a 11 anos':4,
#     '12 anos ou mais':5,
#     'Ignorado':9
# })


In [110]:
# sinasc_apgar5.isna().sum()
sinasc_apgar5.APGAR5.unique()

array([10.,  9.,  7.,  0.,  8.,  4.,  3.,  6.,  5.,  2.,  1.])

In [121]:
# 8)
sinasc_apgar5.loc[(sinasc_apgar5.APGAR5 >= 8) & (sinasc_apgar5.APGAR5 <= 10), 'APGAR5_cat'] = 'normal'
sinasc_apgar5.loc[(sinasc_apgar5.APGAR5 >= 6) & (sinasc_apgar5.APGAR5 <= 7), 'APGAR5_cat'] = 'asfixia leve'
sinasc_apgar5.loc[(sinasc_apgar5.APGAR5 >= 4) & (sinasc_apgar5.APGAR5 <= 5), 'APGAR5_cat'] = 'asfixia moderada'
sinasc_apgar5.loc[(sinasc_apgar5.APGAR5 >= 0) & (sinasc_apgar5.APGAR5 <= 3), 'APGAR5_cat'] = 'asfixia severa'

sinasc_apgar5[['APGAR5','APGAR5_cat']].value_counts()

APGAR5  APGAR5_cat      
9.0     normal              16076
10.0    normal               8393
8.0     normal               1994
7.0     asfixia leve          241
6.0     asfixia leve           79
5.0     asfixia moderada       45
1.0     asfixia severa         24
4.0     asfixia moderada       23
2.0     asfixia severa         18
0.0     asfixia severa         17
3.0     asfixia severa         15
Name: count, dtype: int64

In [124]:
# 9)
snake_case = [
    'local_nascimento',
    'idade_mae',
    'estado_civil_mae',
    'escolaridade_mae',
    'qtd_filhos_vivos',
    'semanas_gestacao',
    'tipo_gravidez',
    'num_consultas',
    'apgar_5min',
    'apgar_5min_cat'
]
sinasc_apgar5.rename(
    columns={
        'LOCNASC':'local_nascimento',
        'IDADEMAE':'idade_mae',
        'ESTCIVMAE':'estado_civil_mae',
        'ESCMAE':'escolaridade_mae',
        'QTDFILVIVO':'qtd_filhos_vivos',
        'GESTACAO':'semanas_gestacao',
        'GRAVIDEZ':'tipo_gravidez',
        'CONSULTAS':'num_consultas',
        'APGAR5':'apgar_5min',
        'APGAR5_cat':'apgar_5min_cat'
    },
    inplace=True,
    errors='raise'
)

sinasc_apgar5.head()

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'APGAR5_cat'],
      dtype='object')